In [1]:
import numpy as np
import pandas as pd
import tensorflow

In [31]:
df_jibun = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\202109_주소DB_전체분\\지번\\지번주소 통합본.csv')
df_jibun

C:\Users\Bestc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,관리번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,지번(본번),지번(부번),지번주소
0,4211010100100010005011491,4211010100,강원도,춘천시,봉의동,0,1,5,강원도 춘천시 봉의동 1-5
1,4211010100100010007011490,4211010100,강원도,춘천시,봉의동,0,1,7,강원도 춘천시 봉의동 1-7
2,4211010100100010008000001,4211010100,강원도,춘천시,봉의동,0,1,8,강원도 춘천시 봉의동 1-8
3,4211010100100010008000001,4211010700,강원도,춘천시,옥천동,0,8,6,강원도 춘천시 옥천동 8-6
4,4211010100100010009011489,4211010100,강원도,춘천시,봉의동,0,1,9,강원도 춘천시 봉의동 1-9
...,...,...,...,...,...,...,...,...,...
8247162,4380036034105910000000001,4380036034,충청북도,단양군,단성면,대잠리,591,0,충청북도 단양군 단성면 대잠리 591
8247163,4380036034200600000000001,4380036034,충청북도,단양군,단성면,대잠리,60,0,충청북도 단양군 단성면 대잠리 60
8247164,4380036034200650000000001,4380036034,충청북도,단양군,단성면,대잠리,65,0,충청북도 단양군 단성면 대잠리 65
8247165,4380036034200850000000001,4380036034,충청북도,단양군,단성면,대잠리,412,1,충청북도 단양군 단성면 대잠리 412-1


In [3]:
# Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks
# 출처 : https://github.com/UKPLab/sentence-transformers
from sentence_transformers import SentenceTransformer, util
model1 = SentenceTransformer('all-mpnet-base-v2')

In [4]:
sentences1 = [juso for juso in df_jibun['지번주소'][:1000]]
sentence_embeddings1 = model1.encode(sentences1, convert_to_tensor = True)

In [7]:
input1 = '갱원도 춘천시 봉의동 1-5'
input_embeddings1 = model1.encode(input1, convert_to_tensor = True)
sentence_embeddings1.shape

torch.Size([1000, 768])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_scores1 = util.pytorch_cos_sim(sentence_embeddings1, input_embeddings1)
cosine_scores_list1 = cosine_scores1.tolist()
cosine_scores_list1.index(max(cosine_scores_list1))
print(f"'{input}'과 가장 유사한 주소(유사도 : {max(cosine_scores_list1)} : '{sentences1[cosine_scores_list1.index(max(cosine_scores_list1))]}'")

'<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x000001E0EF757A90>>'과 가장 유사한 주소(유사도 : [0.9996349811553955] : '강원도 춘천시 봉의동 1-5'


In [9]:
# KoBERT
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
input_ids = torch.LongTensor([[31, 51, 9], [15, 5, 0]])
input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
model, vocab  = get_pytorch_kobert_model()
sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
# pooled_output.shape
vocab
# Last Encoding Layer
pooled_output

using cached model
using cached model


'pooler_output'

In [10]:
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)
sp('한국어 모델을 공유합니다.')

using cached model


['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.']

In [11]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [12]:
tokenizer("한국어 모델을 공유합니다.")

{'input_ids': [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
import torch
from transformers import BertModel
sentences2 = [juso for juso in df_jibun['지번주소']]
model2 = BertModel.from_pretrained('skt/kobert-base-v1')
text = sentences2[0]
inputs = tokenizer.batch_encode_plus([text])
out = model2(input_ids = torch.tensor(inputs['input_ids']),
            attention_mask = torch.tensor(inputs['attention_mask']))
juso_embedding = out.pooler_output
n = 0
for text in sentences2[1:1000]:
    n += 1
    inputs = tokenizer.batch_encode_plus([text])
    out = model2(input_ids = torch.tensor(inputs['input_ids']),
                attention_mask = torch.tensor(inputs['attention_mask']))
    sample = out.pooler_output
    juso_embedding = torch.cat([juso_embedding, sample], dim = 0)
juso_embedding.shape

torch.Size([1000, 768])

In [14]:
juso_embedding.shape

torch.Size([1000, 768])

In [18]:
input2 = '갱원도 춘천시 봉의동 1-5'
inputs = tokenizer.batch_encode_plus([input2])
out = model(input_ids = torch.tensor(inputs['input_ids']),
            attention_mask = torch.tensor(inputs['attention_mask']))
input_embeddings = out.pooler_output

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import util
cosine_scores2 = util.pytorch_cos_sim(juso_embedding, input_embeddings)
cosine_scores2.shape

torch.Size([1000, 1])

In [20]:
cosine_scores_list2 = cosine_scores2.tolist()
cosine_scores_list2.index(max(cosine_scores_list2))
print(f"'{input1}'과 가장 유사한 주소(유사도 : {max(cosine_scores_list1)} : '{sentences1[cosine_scores_list1.index(max(cosine_scores_list1))]}'")
print(f"'{input2}'과 가장 유사한 주소(유사도 : {max(cosine_scores_list2)} : '{sentences2[cosine_scores_list2.index(max(cosine_scores_list2))]}'")

'갱원도 춘천시 봉의동 1-5'과 가장 유사한 주소(유사도 : [0.9996349811553955] : '강원도 춘천시 봉의동 1-5'
'갱원도 춘천시 봉의동 1-5'과 가장 유사한 주소(유사도 : [0.9803658723831177] : '강원도 춘천시 요선동 155-5'
